Instructions for running notebook (on NCAR jupyterhub):  
1) Pull from github: `git clone https://github.com/josephko91/ice3d.git`
2) Make sure you are in the ice3d directory
3) Activate conda module: `module load conda`
4) Download required packages: `conda env create -f cq-conda-env.yaml -n cq`
5) Change the jupyter kernel to cq (or whatever you named it)

*NOTE: check file path for s_code.json

In [76]:
# import packages
import numpy as np
import cadquery as cq
import json
import random
import os
import math
import miniball

# (1) Load inputs

In [2]:
# Read the JSON file as a dictionary
# NOTE: when reading in from JSON, the key is read as a string
json_filepath = '/glade/u/home/joko/ice3d/s_code.json' # change directory as needed
with open(json_filepath, 'r') as json_file:
    s_code_dict = json.load(json_file)

# (2) Source Code

In [3]:
# Helper functions from helper.py
def norm_rows(v):
    """Normalize rows of array v into unit vectors."""
    if np.all(v==0):
        v_unit = np.array([1,0,0])
    else:
        if v.ndim == 1:
            v_norm = np.linalg.norm(v)
            v_unit = v/v_norm
        
        else:
            v_norm = np.linalg.norm(v, axis=1)
            v_unit = v/v_norm[:,None]
    return v_unit

def random_spherical_cap(cone_angle_deg, cone_direction, num_points):
    """
    Generates a desired number of random points on a spherical cap, 
    given a solid angle and cone direction.

    Parameters
    ----------
    cone_angle_deg : float
        Solid angle of the cone used to define the spherical cap, in units of degrees.
    cone_direction : list
        Direction of cone as a list of vector components [x, y, z].
    num_points : int
        Number of points to generate on spherical cap. 
    
    Returns
    ----------
    points_rot 
        List of random points on spherical cap, as numpy array. 
    """
    # generate points on spherical cap centered at north pole
    cone_angle_rad = cone_angle_deg*(np.pi/180)
    z = np.random.uniform(np.cos(cone_angle_rad), 1, num_points)
    phi = np.random.uniform(0, 2*np.pi, num_points)
    x = np.sqrt(1-z**2)*np.cos(phi)
    y = np.sqrt(1-z**2)*np.sin(phi)
    points = np.column_stack((x, y, z))

    # rotate points
    north_vector = np.array([0, 0, 1])
    cone_direction_norm = norm_rows(cone_direction)
    u = norm_rows(np.cross(north_vector, cone_direction_norm)) # rotation axis
    rot = np.arccos(np.dot(cone_direction_norm, north_vector)) # rotation angle in radians
    ux = u[0]
    uy = u[1]
    uz = u[2]
    # define rotation matrix
    r11 = np.cos(rot) + (ux**2)*(1 - np.cos(rot))
    r12 = ux*uy*(1 - np.cos(rot)) - uz*np.sin(rot)
    r13 = ux*uz*(1 - np.cos(rot)) + uy*np.sin(rot)
    r21 = uy*ux*(1 - np.cos(rot)) + uz*np.sin(rot)
    r22 = np.cos(rot) + (uy**2)*(1 - np.cos(rot))
    r23 = uy*uz*(1 - np.cos(rot)) - ux*np.sin(rot)
    r31 = uz*ux*(1 - np.cos(rot)) - uy*np.sin(rot)
    r32 = uz*uy*(1 - np.cos(rot)) + ux*np.sin(rot)
    r33 = np.cos(rot) + (uz**2)*(1 - np.cos(rot))
    rot_mat = np.array([[r11, r12, r13], 
                        [r21, r22, r23], 
                        [r31, r32, r33]])
    
    points_rot = np.matmul(rot_mat, points.T)
    points_rot = points_rot.T   

    return points_rot

def perturb_aspect_ratio(n_arms, f_a_c_limits):
    f_a_c = []
    for i in range(n_arms):
        f_a = random.uniform(f_a_c_limits[0], f_a_c_limits[1])
        f_c = random.uniform(f_a_c_limits[2], f_a_c_limits[3])
        f_a_c.append(f_a)
        f_a_c.append(f_c)
    return f_a_c

def get_cone_angle(n_arms):
    # source: http://neilsloane.com/packings/index.html#I
    min_angles = {4 : 109.4712206, 
                  5 : 90.0000000, 
                  6 : 90.0000000, 
                  7 : 77.8695421,
                  8 : 74.8584922,
                  9 : 70.5287794,
                  10 : 66.1468220}
    cone_angle_deg = min_angles[n_arms]/3 # adjust as needed
    return cone_angle_deg

def perturb_s_code(n_arms, s_code):
    s_code_perturbed = []
    cone_angle_deg = get_cone_angle(n_arms)
    for i in range(n_arms):
        cone_direction = np.array([s_code[3*i], s_code[3*i+1], s_code[3*i+2]])
        points_rot = random_spherical_cap(cone_angle_deg, cone_direction, 1)
        pt = points_rot[0]
        s_code_perturbed.extend(pt)
    return s_code_perturbed


# For each value of n_arm, generate N rosettes using param_list
def create_bullet(a, c, hp, f_a, f_c, workplane):
    # create pyramid
    n_pyr = 6
    ri = a*np.cos(np.radians(30)) # distance between center and edge of hexagon
    theta = 90 - np.degrees(np.arctan(hp/ri))
    pyramid = workplane.polygon(n_pyr, f_a*2*a).extrude(-f_a*hp, taper=theta)
    # create cylinder 
    n_cyl = 6
    cylinder = workplane.polygon(n_cyl, f_a*2*a).extrude(f_c*2*c)
    # create bullet (union)
    bullet = cylinder.union(pyramid)
    return bullet

def calc_r0(f_r0, a, n_arms):
    '''
    linearly interpolate between perscribed limits for r0
    '''
    ymin, ymax = 0.5*a, 1*a
    xmin, xmax = 4, 12
    slope = (ymax-ymin)/(xmax-xmin)
    intercept = ymin - (slope*xmin)
    r0 = slope*(n_arms) + intercept
    r0 = f_r0 * r0 # multiply by perturbation factor
    return r0 

def calc_hp(f_hp, r0, n_arms):
    '''
    linearly interpolate: hp increases with n_arms
    '''
    ymin, ymax = 1*r0, 1.5*r0
    xmin, xmax = 4, 12
    slope = (ymax-ymin)/(xmax-xmin)
    intercept = ymin - (slope*xmin)
    hp = slope*(n_arms) + intercept
    hp = f_hp*hp # multiply by perturbation factot
    return hp

def calc_h0(f_h0, r0):
    '''
    h0 calculate as a perscribed fraction of r0
    '''
    h0 = r0/2
    h0 = f_h0*h0 # multiply by perturbation factor
    return h0

def extract_xyz(s_code):
    '''
    Convert list in format [x1, y1, z1, ..., xn, yn, zn] to separate x, y, z arrays
    '''
    x = []
    y = []
    z = []
    for i in range(0, len(s_code), 3):
        x.append(s_code[i])
        y.append(s_code[i+1])
        z.append(s_code[i+2])
    return x, y, z

def create_ros(params, n_arms, s_code, perturb):
    # unpack parameters
    a, c, f_r0, f_hp, f_h0 = params[0], params[1], params[2], params[3], params[4]
    f_a_min, f_a_max = perturb[0], perturb[1]
    f_c_min, f_c_max = perturb[2], perturb[3]
    perturb_s_code_switch = perturb[4] # 1 for yes, 0 for no
    r0 = calc_r0(f_r0, a, n_arms)
    hp = calc_hp(f_hp, r0, n_arms)
    h0 = calc_h0(f_h0, r0)
    # create sphere
    sphere = cq.Workplane().sphere(r0)
    # create outer shell to "place" bullets on
    # based on spherical code from Sloane et al. 
    r_outer = r0 + hp - h0
    # perturb s_code if necessary 
    if perturb_s_code_switch==1: 
        s_code = perturb_s_code(n_arms, s_code)
    # convert s_code list to outer_coords
    x, y, z = extract_xyz(s_code)
    outer_coords = r_outer*(np.column_stack((x, y, z)))
    # create and collect bullets in list
    bullets = []
    for i in range(len(outer_coords)):
        f_a = random.uniform(f_a_min, f_a_max)
        f_c = random.uniform(f_c_min, f_c_max)
        normal_vector = tuple(outer_coords[i])
        plane = cq.Plane(origin=normal_vector, normal=normal_vector)
        workplane = cq.Workplane(plane)
        bullet = create_bullet(a, c, hp, f_a, f_c, workplane)
        bullets.append(bullet)
    # boolean union to create rosette
    ros = sphere.union(bullets[0])
    for i in range(1, n_arms):
        ros = ros.union(bullets[i])
    return ros

# (3) Main Code

## (a) Generate customized rosette

In [4]:
# Specify parameters
params = [20, 50, 1, 1, 1] # [a, c, f_r0, f_hp, f_h0]
perturb = [0.8, 1.2, 0.8, 1.2, 1] # [f_a_min, f_a_max, f_c_min, f_c_max, perturb_s_code_switch]
n_arms = 10 # choose between 4 and 12
s_code = s_code_dict[str(n_arms)] # extracts spherical code based on n_arms
# create rosette
ros = create_ros(params, n_arms, s_code, perturb)
print(f'rosette surface area: {ros.val().Area()}')
print(f'rosette volume: {ros.val().Volume()}')
ros

rosette surface area: 150651.73489001396
rosette volume: 1236925.380132995


## (b) Generate random rosette

In [4]:
# Specify parameters
a = random.uniform(10, 50)
c_a = random.uniform(1.25, 10)
c = c_a * a
f_r0 = random.uniform(0.8, 1.2)
f_hp = random.uniform(0.8, 1.2)
f_h0 = random.uniform(0.8, 1.2)
params = [a, c, f_r0, f_hp, f_h0] 
perturb = [0.8, 1.2, 0.8, 1.2, 1] # [f_a_min, f_a_max, f_c_min, f_c_max, perturb_s_code_switch]
n_arms = random.randint(4, 10) # choose between 4 and 10
s_code = s_code_dict[str(n_arms)] # extracts spherical code based on n_arms
# create random rosette
ros = create_ros(params, n_arms, s_code, perturb)
print(f'rosette surface area: {ros.val().Area()}')
print(f'rosette volume: {ros.val().Volume()}')
ros

rosette surface area: 133739.4802779836
rosette volume: 898239.7361460286


In [36]:
r0 = calc_r0(f_r0, a, n_arms)
r0

12.170144274232829

In [47]:
origin = cq.Vertex.makeVertex(0,0,0)
origin.Center()

Vector: (0.0, 0.0, 0.0)

In [79]:
# get vertices of ros
verts = ros.vertices() # list of vertices
# for v in verts:
#     # print(v)
#     print(type(v))
#     print(v.Center())
# filter verts (everything less than int(r0)
threshold = math.ceil(r0)
origin = cq.Vertex.makeVertex(0,0,0)
filtered_verts = [v for v in verts if v.distance(origin) > threshold]
for v in filtered_verts:
    print(v.Center())

Vector: (16.710447538971245, -2.2762694016230576, -16.803910631806126)
Vector: (19.239851172449704, 9.574296339109308, -10.244649741992472)
Vector: (21.53979846780609, 9.574296339109317, 3.3409285296783935)
Vector: (21.310342129684013, -2.276269401623056, 10.367245911535608)
Vector: (20.01945578581444, -8.324229987811666, 7.019713632990584)
Vector: (18.57862323523068, -8.030693551161123, 5.6920050569092595)
Vector: (15.42584899896234, -9.687294094116563, 2.000665133032606)
Vector: (17.623966938840663, -14.126835142355423, -3.0261401264554295)
Vector: (16.48099120084917, -14.126835142355425, -9.777593249948914)
Vector: (-11.44257793505256, 17.707889854974656, -12.992053537524157)
Vector: (3.40682081284568, 17.70788985497466, -16.97408662847009)
Vector: (9.977434882313082, 4.8083770069323934, -22.15007250024176)
Vector: (1.6986502038822422, -8.091135841109871, -23.34402528106749)
Vector: (-13.150748544015997, -8.09113584110988, -19.36199219012156)
Vector: (-17.019920800638555, 4.37429116

In [80]:
verts_list = [v for v in verts]
print(f'unfiltered: {len(verts_list)}')
print(f'filtered: {len(filtered_verts)}')

unfiltered: 131
filtered: 91


In [81]:
filter_verts_list = [v.Center() for v in filtered_verts]
test_vert = filter_verts_list[0]
test_vert.toTuple()

(16.710447538971245, -2.2762694016230576, -16.803910631806126)

In [82]:
verts_list = np.asarray([list(v.Center().toTuple()) for v in verts])
verts_list[:5]

array([[ 7.45206412e-16, -1.82522929e-31, -1.21701443e+01],
       [ 1.12244511e+01, -2.74919762e-15, -4.70362722e+00],
       [ 1.14884805e+01,  1.58823718e+00, -3.68845899e+00],
       [ 1.20623707e+01,  1.58823718e+00, -2.98542182e-01],
       [ 1.21471996e+01, -2.97520581e-15,  7.46963499e-01]])

In [83]:
filtered_verts_list = np.asarray([list(v.Center().toTuple()) for v in filtered_verts])
filtered_verts_list[:5]

array([[ 16.71044754,  -2.2762694 , -16.80391063],
       [ 19.23985117,   9.57429634, -10.24464974],
       [ 21.53979847,   9.57429634,   3.34092853],
       [ 21.31034213,  -2.2762694 ,  10.36724591],
       [ 20.01945579,  -8.32422999,   7.01971363]])

In [84]:
# compute miniball of verts and filtered verts for validation
def calc_mbs(points):
    """
    Calculate minimal bounding sphere (mbs)
    """
    mbs = {} # store attributes of sphere as dict

    # use miniball algorithm to find bounding sphere
    mesh_points = np.asarray(points)
    unique_pts = np.unique(mesh_points, axis=0)
    c, r2 = miniball.get_bounding_ball(unique_pts)
    r = np.sqrt(r2) # r2 = radius squared, r = radius

    mbs['c'] = c # center coordinates as np array
    mbs['r'] = r # radius of sphere as float
    mbs['v'] = (4/3)*np.pi*(r**3)

    return mbs
mbs_unfiltered = calc_mbs(verts_list)
mbs_filtered = calc_mbs(filtered_verts_list)
print(f'unfiltered r: {mbs_unfiltered['r']}')
print(f'filtered r: {mbs_filtered['r']}')

unfiltered r: 240.18076846295924
filtered r: 240.18076846295924


In [85]:
%%time
# time to calculate mbs (unfiltered)
for i in range(1000):
    mbs_unfiltered = calc_mbs(verts_list)

CPU times: user 36.5 s, sys: 64.1 ms, total: 36.6 s
Wall time: 38.4 s


In [86]:
%%time
# time to calculate mbs (filtered)
for i in range(1000):
    mbs_filtered = calc_mbs(filtered_verts_list)

CPU times: user 25.9 s, sys: 60.2 ms, total: 26 s
Wall time: 28.1 s


In [88]:
%%time
# time to filter points
for i in range(1000):
    filtered_verts = [v for v in verts if v.distance(origin) > threshold]

CPU times: user 1.45 s, sys: 3.93 ms, total: 1.45 s
Wall time: 1.48 s


## (c) Save rosette as STL (mesh format)

In [ ]:
# if you want to save file
save_dir = '/glade/u/home/joko/ice3d/output'
save_filename = 'ros-test.stl'
save_filepath = os.path.join(save_dir, save_filename)
cq.exporters.export(ros, save_filepath)